In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Define the search query
location = input("Enter the location: ")
min_price = input("Enter the minimum price: ")
max_price = input("Enter the maximum price: ")
min_area = input("Enter the minimum area: ")
max_area = input("Enter the maximum area: ")
distance = input("Enter the maximum distance from location in km: ")
listing_type = input("Enter listing type (mieszkanie/dom/dzialka): ")
offer_type = input("Enter offer type (sprzedaz/wynajem): ")

# Start the Chrome driver and maximize the window
driver = webdriver.Chrome()
driver.maximize_window()

# Set the variable to check if the banner has been closed
is_banner_closed = False
listings_combined =[]


# Iterate through the pages and open each page in a new window
for page in range(1, 5): #page number
    # Construct the URL
    url =f"https://www.otodom.pl/pl/oferty/{offer_type}/{listing_type}/{location}?priceMin={min_price}&priceMax={max_price}&areaMin={min_area}&areaMax={max_area}&distanceRadius={distance}&page={page}&limit=72"

    # Open the URL in a new window
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(url)

    time.sleep(3)

    # Close Cookies banner
    if not is_banner_closed:
        try:
            cookie_banner = driver.find_element(By.ID,'onetrust-accept-btn-handler')
            cookie_banner.click()
            is_banner_closed = True
        except:
            pass

    # Scroll page down till the end
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Wait for the results to load
    #WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@role='navigation']")))


    # Scroll down to the end of the page
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(driver.page_source)

    # Find all the relevant information

    listings = soup.find_all("li", {'class': 'css-iq9jxc e1n6ljqa1'})
    listings_combined.append({'page': page, 'listings': listings})

# Close all windows and quit the driver
driver.quit()    

In [22]:
listings_combined

[{'page': 1,
  'listings': [<li class="css-iq9jxc e1n6ljqa1" data-cy="listing-item"><a class="css-1up0y1q e1n6ljqa16" data-cy="listing-item-link" href="/pl/oferta/mieszkanie-2-pokoje-zielona-bialoleka-ID4kyRy"><aside class="css-qou0ww e1n6ljqa3"><picture><source data-cy="listing-item-image-source" media="(max-width: 768px)" srcset="https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6Ijc5Z29xbHJleWVyMS1BUEwiLCJ3IjpbeyJmbiI6ImVudmZxcWUxYXk0azEtQVBMIiwicyI6IjE0IiwicCI6IjEwLC0xMCIsImEiOiIwIn1dfQ.zbUCCdDEaXYpr_Lq88zREjLf2a2GLaZCjpt5NBqqE-w/image;s=655x491;q=80"><source data-cy="listing-item-image-source" media="(min-width: 768px)" srcset="https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6Ijc5Z29xbHJleWVyMS1BUEwiLCJ3IjpbeyJmbiI6ImVudmZxcWUxYXk0azEtQVBMIiwicyI6IjE0IiwicCI6IjEwLC0xMCIsImEiOiIwIn1dfQ.zbUCCdDEaXYpr_Lq88zREjLf2a2GLaZCjpt5NBqqE-w/image;s=1280x1024;q=80"><img alt="Mieszkanie 2 pokoje - Zielona Białołęka" class="css-s6l5j9 e1xjv2fs0" height="210" loading="lazy" src="https://ireland.

In [23]:
#this gets the Top 13 results only
data = []

for i in listings_combined:
    for record in listings:
    # Extract the link to the record
        link = record.find("a", {'class':'css-1up0y1q e1n6ljqa16'}).get("href")
        link = "https://www.otodom.pl/"+link

    # Extract the title
        property_title = record.find("h3", {'class' :'css-1mmyqk7 e1n6ljqa6'}).get("title")

    # Extract the address
        address =record.find("p", {'class' :'css-14aokuk e1n6ljqa7'}).text.strip()    

    # Extract the number of rooms, area, and price
        details = record.find_all("span", {'class':'css-1ntk0hg ei6hyam1'})
        rooms = ""
        area = ""
        price = ""
        for detail in details:
            if "pokoje" in detail.text:
                rooms = detail.text.strip().replace(" pokoje", "")
            elif "m²" in detail.text:
                area = detail.text.strip().replace(" m²", "")
            elif "zł/mc" in detail.text:
                price = detail.text.strip().replace("zł/mc", "")

        # Append the attributes to the data list
        data.append({
            "Link": link,
            "Property": property_title,
            "Address ": address,
            "Rooms": rooms,
            "Area (m²)": area,
            "Price (zł/mc)": price
        })



# Create a dataframe from the data list
df = pd.DataFrame(data)

In [24]:
df

,Link,Property,Address,Rooms,Area (m²),Price (zł/mc)
0,https://www.otodom.pl//pl/oferta/3-pokoje-na-z...,3 pokoje na zielonych Bielanach /osiedle PIASKI/,"Warszawa, Bielany, Piaski, ul. Władysława Bron...",3,50.61,3200
1,https://www.otodom.pl//pl/oferta/atrakcyjne-um...,"Atrakcyjne, umeblowane mieszkanie","Warszawa, Bielany, Młociny, Encyklopedyczna",2,58,3300
2,https://www.otodom.pl//pl/oferta/nowe-2-pokoje...,"Nowe, 2 pokoje, Mokotów, bezpośrednio","Warszawa, Mokotów, ul. Bernardyńska",2,40,3000
3,https://www.otodom.pl//pl/oferta/do-wynajecia-...,Do wynajęcia przestronna kawalerka (32m) Tarch...,"Warszawa, Białołęka, Tarchomin, ul. Pasłęcka",,32,2000
4,https://www.otodom.pl//pl/oferta/ursynow-przy-...,"Ursynów przy metrze, klimatyzacja, garaż","Warszawa, Ursynów, ul. Dereniowa",2,50,3600
...,...,...,...,...,...,...
295,https://www.otodom.pl//pl/oferta/funkcjonale-m...,Funkcjonale mieszkanie 3 pokojowe w Centrum,"Warszawa, Wola, Mirów, ul. Chłodna",3,53,4000
296,https://www.otodom.pl//pl/oferta/klimatyczne-c...,"Klimatyczne, ciche mieszkanie na Kabatach.","Warszawa, Ursynów, Kabaty, ul. Kazimierza Jeże...",,37,2250
297,https://www.otodom.pl//pl/oferta/widok-14-piet...,"widok 14 piętro,garaż podziemny,metro Daszyńsk...","Warszawa, Wola, ul. Giełdowa 4B",2,53,3900
298,https://www.otodom.pl//pl/oferta/blisko-s8-tra...,"blisko s8, tramwaj - nowy blok","Warszawa, Praga-Północ, ul. Aleksandra Kotsisa",,30.5,2700
